# 데이터 분석 코드

# 응급실 데이터

In [6]:
import pandas as pd
import numpy as np
import warnings
from haversine import haversine
warnings.filterwarnings('ignore')

# 부산광역시 
# 응급실 실시간 데이터
pd.set_option('display.max_columns', None)
url = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire?serviceKey=Zv%2FutM8zPV4Cpby0tQi%2FtiDmHPhc19Zr67HNB4XUKyLt89VHIxcmVtAdRSWES5ve3JQGe4P3OL6dyL6vChTt8Q%3D%3D&STAGE1=%EB%B6%80%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C&pageNo=1&numOfRows=100'
df1 = pd.read_xml(url, xpath='.//item')
df1_c = df1.copy()
df2 = df1_c[['dutyName','hpid','hvec','hvoc', 'dutyTel3']]
df2_c = df2.rename(columns={'dutyName':'병원명', 'hpid':'병원코드', 'hvoc':'수술실수', 'dutyTel3':'응급실연락처','hvec':'응급실수'})
# df2_c

# # 중증질환자 수용가능정보 실시간 데이터
url = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getSrsillDissAceptncPosblInfoInqire?serviceKey=Zv%2FutM8zPV4Cpby0tQi%2FtiDmHPhc19Zr67HNB4XUKyLt89VHIxcmVtAdRSWES5ve3JQGe4P3OL6dyL6vChTt8Q%3D%3D&STAGE1=%EB%B6%80%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C&pageNo=1&numOfRows=100'
df3 = pd.read_xml(url, xpath='.//item')
df3.columns = ['병원명','병원코드','뇌출혈','신생아','중증화상','뇌경색','응급실','심근경색','복부손상','사지접합','응급내시경','응급투석', '조산산모','정신질환자']
df3.drop(['병원명','정신질환자','응급실'],axis=1, inplace=True)
# df3

# # 병원 위치 데이터
url = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytListInfoInqire?serviceKey=Zv%2FutM8zPV4Cpby0tQi%2FtiDmHPhc19Zr67HNB4XUKyLt89VHIxcmVtAdRSWES5ve3JQGe4P3OL6dyL6vChTt8Q%3D%3D&Q0=%EB%B6%80%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C&pageNo=1&numOfRows=100'
df4 = pd.read_xml(url, xpath='.//item')
df5 = df4[['phpid','dutyAddr','wgs84Lat','wgs84Lon']]
df5_c = df5.rename(columns={'phpid':'병원코드','dutyAddr':'병원주소','wgs84Lat':'위도','wgs84Lon':'경도'})
# df5_c

# # 총 데이터 merge
dfx = pd.merge(df2_c, df5_c, on='병원코드')
dfa = pd.merge(dfx, df3, on='병원코드', how='outer')
# # dfa

# # 병원별 전체병상수, 응급실 포화도/상태 추가
er = pd.read_excel('er1.xlsx')
er.drop(['Unnamed: 0','병원명'], axis=1, inplace=True)
df_er = pd.merge(dfa, er, on='병원코드')
df_er['응급실포화도'] = (1-df_er['응급실수']/df_er['전체병상수'])*100
df_er['응급실포화상태'] = df_er['응급실포화도']
df_er['응급실포화상태'] = pd.cut(df_er['응급실포화상태'] ,bins=[0,30,60,90,np.inf],labels=["원활","보통","혼잡","매우혼잡"],right=False) #라벨 설정
df_er.insert(6, column= "거리", value= 0)

# 부산역을 현위치(A)로 설정
A = (35.11492,  129.04251)
for i in range(df_er.shape[0]):
    B = (df_er["위도"][i], df_er["경도"][i])
    df_er["거리"][i] = round(haversine(A, B), 3)

df_er

# csv로 저장
# df_er.to_csv('hospital.csv', encoding='cp949', index=False)

,병원명,병원코드,응급실수,수술실수,응급실연락처,병원주소,거리,위도,경도,뇌출혈,신생아,중증화상,뇌경색,심근경색,복부손상,사지접합,응급내시경,응급투석,조산산모,전체병상수,응급실포화도,응급실포화상태
0,(의)서일의료재단기장병원,A1200028,3,3,051-723-2119,부산광역시 기장군 기장읍 대청로72번길 6,20.770,35.236029,129.216492,N,N,N,N,N,N,N,Y,N,N,10,70.000000,혼잡
1,동남권원자력의학원,A1200015,6,6,051-720-5911,부산광역시 기장군 장안읍 좌동길 40,29.345,35.321419,129.243649,N,N,N,N,N,N,N,Y,N,N,10,40.000000,보통
2,부산성모병원(재단법인천주교부산교구유지재단),A1200013,4,7,051-933-7119,부산광역시 남구 용호로232번길 25-14 (용호동),6.086,35.110457,129.109192,Y,Y,N,Y,Y,Y,N,Y,Y,Y,19,78.947368,혼잡
3,좋은문화병원,A1200026,4,0,051-630-0909,부산광역시 동구 범일로 119 (범일동),3.240,35.140724,129.059064,N,Y,N,N,N,N,N,N,N,Y,11,63.636364,혼잡
4,의료법인정화의료재단김원묵기념봉생병원,A1200017,8,8,051-664-4119,"부산광역시 동구 중앙대로 401 (좌천동, 봉생병원)",1.918,35.130858,129.050571,Y,N,N,Y,Y,N,N,Y,Y,N,11,27.272727,원활
5,동래봉생병원,A1200011,5,5,051-520-5600,부산광역시 동래구 안연로109번길 27 (안락동),10.343,35.196946,129.096167,Y,N,N,Y,Y,N,N,N,N,N,10,50.000000,보통
6,대동병원,A1200018,9,6,051-550-9391,부산광역시 동래구 충렬대로 187 (명륜동),10.510,35.204273,129.080215,Y,N,N,N,Y,Y,N,N,N,N,21,57.142857,보통
7,의료법인광혜의료재단광혜병원,A1200009,4,3,051-590-3399,"부산광역시 동래구 충렬대로 96 (온천동, 광혜병원)",10.568,35.207033,129.071136,N,N,N,N,N,Y,N,N,N,N,10,60.000000,혼잡
8,의료법인온그룹의료재단온종합병원,A1200020,14,9,051-607-0139,"부산광역시 부산진구 가야대로 721, 719, 767 (당감동)",4.825,35.157877,129.049984,Y,N,N,Y,Y,Y,N,Y,Y,N,30,53.333333,보통
9,인제대학교부산백병원,A1200001,21,17,051-890-5995,"부산광역시 부산진구 복지로 75, 진사로83번길 81, 1층(일부), 3층 (개금동)",4.034,35.146454,129.020572,Y,Y,N,Y,Y,Y,Y,Y,Y,Y,35,40.000000,보통


# 질환 예측

In [7]:
# 질환별 증상데이터 + 전처리
d1 = pd.read_excel('fd_2.xlsx')
d1.drop('Unnamed: 0', axis=1, inplace=True)

# 이름이 같은 증상 제거
d1['협심증'] = d1['협심증'] + d1['협심증.1']
d1['호흡음 감소'] = d1['호흡음 감소.1'] + d1['호흡음 감소']
d1['흉부 압박'] = d1['흉부 압박.1'] + d1['흉부 압박']
d1['운동 시 호흡곤란'] = d1['운동 시 호흡곤란.1'] + d1['운동 시 호흡곤란']
d1['안면마비'] = d1['안면마비.1'] + d1['안면마비']
d1['피로'] = d1['피로.1'] + d1['피로']
d1['일반적인 불편함'] = d1['일반적인 불편함.1'] + d1['일반적인 불편함']
d1['경정맥 팽창'] = d1['경정맥 팽창.1'] + d1['경정맥 팽창']
d1['야간 땀'] = d1['야간 땀.1'] + d1['야간 땀']
d1['흉막 통증'] = d1['흉막 통증.1'] + d1['흉막 통증']
d1['비생산적인 기침'] = d1['비생산적인 기침.1'] + d1['비생산적인 기침']
d1['손의 마비'] = d1['손의 마비.1'] + d1['손의 마비']
d1['통증 가슴'] = d1['통증 가슴.1'] + d1['통증 가슴']
d1['압력 가슴'] = d1['압력 가슴'] + d1['압력 가슴.1']
d1['생산적인 기침'] = d1['생산적인 기침.1'] + d1['생산적인 기침']
d1['호흡 곤란'] = d1['호흡 곤란.1'] + d1['호흡 곤란']
d1['말이 어눌해졌다'] = d1['말이 어눌해졌다'] + d1['말이 어눌해졌다.1']
d1['발한 증가'] = d1['발한 증가.1'] + d1['발한 증가.1']
d1['노란 가래'] = d1['노란 가래.1'] + d1['노란 가래']
colsx = ['협심증.1', '호흡음 감소.1', '흉부 압박.1', '운동 시 호흡곤란.1', '안면마비.1', '피로.1', '일반적인 불편함.1', '경정맥 팽창.1', '현기증.1', '야간 땀.1', '비생산적인 기침.1', '손의 마비.1', '통증 가슴.1', '흉막 통증.1', '압력 가슴.1', '생산적인 기침.1', '호흡 곤란.1', '말이 어눌해졌다.1', '발한 증가.1', '노란 가래.1']
d1.drop(colsx, axis=1, inplace=True)

# 질환에 포함되지 않는 증상들 제거
col = d1.columns
a = []
for x in col:
    if d1[[x]].value_counts()[0]==24:
        a.append(x)
        
d2 = d1.drop(a, axis=1)
d2

# d2.to_excel('disease.xlsx', index=False, encoding='utf-8')

,수술,질병,스탈리의 선,복부 급성,복부 압통,비정상적인 감각,농양 박테리아,결석 발견,공기 유체 레벨,협심증,신경성 식욕 부진증,실어증,미열의,자세고정불능증,무력증,무증상,영기,정전,서맥,호흡음 감소,소문,심비대증,숨을 고르다,흉부 압박,가슴 불편,반흔화,간헐성 경련,서투름,의식이 맑다,변비,기침,쇠약,체중 감소,설사,어려움,현기증,어지러운 주문,졸음,구음 장애,호흡곤란,운동 시 호흡곤란,반상 출혈,홍진,피로,추체외로징후,안면마비,떨어지다,피로도,열,플레어,공허,초점 발작,프레미투스,거품 가래,일반적인 불편함,일반적인 불안정,중력 0,구아이악 포지티브,출혈,두통,편마비,혈역학적으로 안정,과오조,고칼륨혈증,저운동증,저나트륨혈증,저혈압,인지 장애,취함,경정맥 팽창,힘든 호흡,좌심방 비대,장애,신체 구조의 질량,종격동 이동,전이성 병변,근육통,메스꺼움,목 경직,야간 땀,비생산적인 기침,얼어서 고움,손의 마비,핍뇨,기좌 호흡,통증,통증 가슴,복통,하지 통증,심계항진,범수축기 잡음,마비시키다,부전 마비,환자 비준수,심낭 마찰 문지름,수명,흉막 통증,다음증,자세,q 파의 존재,압력 가슴,프로드롬,생산적인 기침,전립선,맥동 역설,랄레,장황한 연설,홍반,휴식 통증,엄격함 - 온도 관련 관찰,방 회전,좌골 신경통,발작,쏘는 고통,호흡 곤란,동리듬,졸린,코를 골다,만지면 아프다,말이 어눌해졌다,화농성 가래,st 세그먼트 함몰,st 세그먼트 표고,따가움,대변 ​​색상 노란색,무감각,땀,발한 증가,부종,졸도,수축기 박출 잡음,수축기 잡음,t파 반전,빈호흡,두껍게 하다,긴장 발작,비협조,무반응,소변 주저,선회,시력이 흐려짐,구토,살찌 다,쌕쌕거림,노란 가래
0,심근경색,고혈압 질환,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,심근경색,관상동맥경화증,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,심근경색,관상 동맥 심장 질환,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,심근경색,실패 심장 울혈,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,뇌출혈,사고 뇌혈관,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,심근경색,심근 경색증,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,2,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
6,심근경색,심부정맥 혈전증,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,심근경색,심근병증,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
8,뇌경색,말초 혈관 질환,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
9,심근경색,승모판 부전,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0


# 모델링 (프로토타입)

In [8]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

x = d2.drop(['질병','수술'], axis=1)
y = d2['수술']

x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.01,random_state=1234)

model1 = DecisionTreeClassifier()
model1.fit(x_train, y_train)
pred1 = model1.predict(x_test)
print(accuracy_score(pred1,y_test))

# print(x_test)
# print(pred1)

1.0


In [9]:
model2 = DecisionTreeClassifier()
model2.fit(x,y)
pred2 = model2.predict(x)
model2.score(x,y)

1.0

In [10]:
# 증상으로 예측 예시
patient = [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
patient = np.array(patient)
patient = patient.reshape(1, 143)
pred3 = model2.predict(patient)[0]
print("="*17)
print("예측질병:", pred3)
print("="*17)
for i in df_er.columns:
    if pred3 == i:
        hospital = df_er.loc[df_er[i] == "Y", :]
hospital = hospital[(hospital['응급실수']>0)&(hospital['수술실수']>0)]
hospital = hospital.reset_index(drop=True)
hospital

예측질병: 심근경색


,병원명,병원코드,응급실수,수술실수,응급실연락처,병원주소,거리,위도,경도,뇌출혈,신생아,중증화상,뇌경색,심근경색,복부손상,사지접합,응급내시경,응급투석,조산산모,전체병상수,응급실포화도,응급실포화상태
0,부산성모병원(재단법인천주교부산교구유지재단),A1200013,4,7,051-933-7119,부산광역시 남구 용호로232번길 25-14 (용호동),6.086,35.110457,129.109192,Y,Y,N,Y,Y,Y,N,Y,Y,Y,19,78.947368,혼잡
1,의료법인정화의료재단김원묵기념봉생병원,A1200017,8,8,051-664-4119,"부산광역시 동구 중앙대로 401 (좌천동, 봉생병원)",1.918,35.130858,129.050571,Y,N,N,Y,Y,N,N,Y,Y,N,11,27.272727,원활
2,동래봉생병원,A1200011,5,5,051-520-5600,부산광역시 동래구 안연로109번길 27 (안락동),10.343,35.196946,129.096167,Y,N,N,Y,Y,N,N,N,N,N,10,50.000000,보통
3,대동병원,A1200018,9,6,051-550-9391,부산광역시 동래구 충렬대로 187 (명륜동),10.510,35.204273,129.080215,Y,N,N,N,Y,Y,N,N,N,N,21,57.142857,보통
4,의료법인온그룹의료재단온종합병원,A1200020,14,9,051-607-0139,"부산광역시 부산진구 가야대로 721, 719, 767 (당감동)",4.825,35.157877,129.049984,Y,N,N,Y,Y,Y,N,Y,Y,N,30,53.333333,보통
5,인제대학교부산백병원,A1200001,21,17,051-890-5995,"부산광역시 부산진구 복지로 75, 진사로83번길 81, 1층(일부), 3층 (개금동)",4.034,35.146454,129.020572,Y,Y,N,Y,Y,Y,Y,Y,Y,Y,35,40.000000,보통
6,학교법인)동의병원,A1200019,7,1,051-850-8777,"부산광역시 부산진구 양정로 62, 지상2ㆍ3ㆍ10층 각 일부/ 지하1ㆍ지상1ㆍ지상4...",6.863,35.169922,129.076744,Y,N,N,Y,Y,N,N,Y,Y,N,17,58.823529,보통
7,구포성심병원,A1200045,2,4,051-330-2090,부산광역시 북구 낙동대로 1786 (구포동),11.085,35.209710,129.004751,Y,N,N,N,Y,Y,N,Y,Y,N,10,80.000000,혼잡
8,의료법인은성의료재단좋은삼선병원,A1200005,6,6,051-310-9416,부산광역시 사상구 가야대로 326 (주례동),5.045,35.150568,129.008198,Y,N,N,Y,Y,Y,Y,Y,Y,N,20,70.000000,혼잡
9,부산대학교병원,A1200002,5,26,051-240-7776,부산광역시 서구 구덕로 179 (아미동1가),2.620,35.101054,129.019222,Y,Y,N,Y,Y,Y,Y,Y,Y,Y,32,84.375000,혼잡


In [11]:
# 프로토타입용 데이터 확인
oo = pd.read_csv("self.csv")
oo

,수술명,붓기,호흡곤란,기력없음,어지럼증,가슴답답함,저혈압,두통,의식저하,구토,고혈압,마비,감각이상,구역질,식은땀,발열,흉통,복통,이물질,열상,저체온,맥박이상,쇼크
0,응급투석,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,뇌출혈,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0
2,뇌경색,0,1,0,1,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0
3,심근경색,0,1,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0
4,응급내시경,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0
5,중증화상,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1
